
# Prácticas Circuitos Cuánticos con Qiskit


###  <img src="https://cdn-icons-png.flaticon.com/512/1373/1373613.png" width="20" height="20" /><font face="Roboto" color='#76b900'> Manuel Benavent Lledó</font> <font face="Roboto" size=3 color='#404040'>(*mbenavent@dtic.ua.es*)</font>

### <img src="https://cdn-icons-png.flaticon.com/512/1373/1373613.png" width="20" height="20" /> <font face="Roboto" color='#76b900'>David Mulero Pérez</font> <font face="Roboto" size=3 color='#404040'>(*dmulero@dtic.ua.es*)</font>

El objetivo de esta práctica es comprender las puertas cuánticas básicas así como los conceptos base de la computación cuántica: la superposicón y el entrelazamiento. Para ello se utilizará la librería de Python, Qiskit, que nos permite la simulación y ejecución de circuitos de unos pocos qubits en un ordenador cuántico de IBM.



## Preparación del entorno
Instalamos Qiskit e importamos las librerías necesarias.

In [1]:
%pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━

In [2]:
# %%capture
# %matplotlib inline
import numpy as np
from qiskit import Aer, QuantumCircuit, execute, assemble, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import QasmSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options
from IPython.display import display, Math, Latex
import math

### Selección del backend
Qiskit nos propociona la opción ejecutar nuestras soluciones sobre un simulador o sobre alguno de los proveedores de hardware, es decir, en un ordenador cuántico físico. Aunque en esta práctica no es necesario medir el estado de los qubits, a continuación incluimos el código para la selección de cada tipo de backend. A continuación podemos ver un ejemplo básico utilizando la simulación.

In [3]:
sim = Aer.get_backend('aer_simulator')
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()
qc.draw()

┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

Para realizar la ejecución en un ordenador cuántico de IBM hay que seguir los pasos de este [enlace](https://quantum-computing.ibm.com/services/programs/docs/runtime/programs/start) y crear nuestra cuenta.
Mediante este "Hello World" podemos comprobar la conexión de nuestra cuenta de IBM. Desde la interfaz en nuestra cuenta también podemos ejecutar diferentes circuitos mediante interfaz gráfica o introduciendo código Qiskit.

### Backend IBM

In [4]:
service = QiskitRuntimeService(channel="ibm_quantum", token='MY_IBM_API_TOKEN')

RequestsApiError: ignored

In [ ]:
program_inputs = {'iterations': 1}
options = {"backend_name": "ibmq_qasm_simulator"}
job = service.run(program_id="hello-world",
                options=options,
                inputs=program_inputs
                )
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

El siguiente ejemplo muestra la ejecución para un circuito con una puerta Hadamard.

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

options = Options()
options.optimization_level = 2
options.resilience_level = 0

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    sampler = Sampler(session=session, options=options)
    job = sampler.run(qc)
    result = job.result()

display(qc.draw())
print(f" > Quasi probability distribution: {result.quasi_dists[0]}")
print(f" > Metadata: {result.metadata[0]}")

## Qiskit
Se trata de una librería open-source para desarrollo cuántico. Para conocer más información se recomienda visitar la [documentación](https://qiskit.org/documentation/index.html), además de los [tutoriales](https://qiskit.org/documentation/tutorials.html) y el [libro de texto](https://qiskit.org/textbook/preface.html), en el cual se incluyen tanto conceptos prácticos como las bases teóricas.

### Ejemplo 1: Circuito con una puerta X (NOT)
Este circuito implementa lo equivalente a un NOT clásico.

In [5]:
# Set the intial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2) # |0>

# Create circuit with a X (NOT) gate
qc = QuantumCircuit(1) # takes 1 qubit
qc.x(0) # apply X to qubit 0

# Print the circuit
print(qc) # qc.draw('mlp') outside Colab to get nicer circuits

# Evolve the state by the quantum circuit
state = state.evolve(qc)

# Obtain probabilities
print(state.probabilities_dict())

# Print state in Latex style
state.draw('latex')

   ┌───┐
q: ┤ X ├
   └───┘
{'1': 1.0}


<IPython.core.display.Latex object>

### Ejemplo 2: Circuito con una puerta H (Hadamard)
En este circuito definimos un qubit en estado de superposición con probabilidad del 50% para cada estado utilizando la puerta Hadamard.

In [6]:
# Set the intial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2) # |0>

# Create circuit with a Hadamard gate (superposition of qubit)
qc = QuantumCircuit(1) # takes 1 qubit
qc.h(0) # apply H to qubit 0

# Print the circuit
print(qc) # qc.draw('mlp') outside Colab to get nicer circuits

# Evolve the state by the quantum circuit
state = state.evolve(qc)

# Obtain probabilities
print(state.probabilities_dict())

# Print state in Latex style
state.draw('latex')

   ┌───┐
q: ┤ H ├
   └───┘
{'0': 0.4999999999999999, '1': 0.4999999999999999}


<IPython.core.display.Latex object>

### Ejemplo 3: Circuito con puerta controlada
En este ejemplo podemos ver como utilizar un bit como control de otra, concretamente para una puerta X, es decir, una puerta CNOT clásica. Se puede utilizar también para el resto de puertas cuánticas poniendo una 'c' delante (ver [documentación](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html)).

In [7]:
# Set the intial state of the simulator to the ground state using from_int
state = Statevector.from_int(2, 2**2) # |10>

# Create circuit with a CNOT gate
qc = QuantumCircuit(2) # takes 2 qubits
# Apply controlled gate: note that qiskit uses little endian notation, a higher index implies more relevance
qc.cnot(1,0) # also qx()

# Print the circuit
print(qc) # qc.draw('mlp') outside Colab to get nicer circuits

# Evolve the state by the quantum circuit
state = state.evolve(qc)

# Obtain probabilities
print(state.probabilities_dict())

# Print state in Latex style
state.draw('latex')

     ┌───┐
q_0: ┤ X ├
     └─┬─┘
q_1: ──■──
          
{'11': 1.0}


<ipython-input-7-bd5c108f1993>:7: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.cnot()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use QuantumCircuit.cx as direct replacement.
  qc.cnot(1,0) # also qx()


<IPython.core.display.Latex object>

## Ejercicios
Resuelve los siguientes ejercicios completando únicamente los trozos de código indicados, **sin modificar** el código proporcionado.

Además de las puertas vistas en los ejemplos se pueden encontrar otras muchas puertas en la documentación de Qiskit además de poder hacer combinaciones de las mismas para por ejemplo utilizar más de un bit de control. Algunos de estos casos se encuentran estandarizados como la [puerta Toffoli](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CCXGate.html), que puede ser de utilidad en alguno de los ejercicios.

### Ejercicio 1
El entrelazamiento (entanglement) es cuando dos o más qubits tienen propiedades unidas. Por ejemplo en el estado $|00\rangle + |11\rangle$, el valor del segundo qubit viene determinado por el valor del primero.
En caso de no estar entrelazados el estado sería $|00\rangle + |01\rangle + |10\rangle + |11\rangle$

Completa el código a continuación para que la salida de los dos qubits quede entrelazada. Estado esperado: $\frac{\sqrt2}{2}|00\rangle + \frac{\sqrt2}{2}|11\rangle$

In [8]:
state = Statevector.from_int(0, 2**2)
qc = QuantumCircuit(2)

## |00>
#========================

# Aplicar la puerta Hadamard al primer qubit
qc.h(0)

# Aplicar la puerta CNOT con el primer qubit como control y el segundo como objetivo
qc.cx(0, 1)

# Evolucionar el Statevector según el circuito cuántico
state = state.evolve(qc)

#========================
state = state.evolve(qc)
print(qc)
print(state.probabilities_dict())
state.draw('latex')

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
{'00': 0.2499999999999999, '01': 0.2499999999999999, '10': 0.2499999999999999, '11': 0.2499999999999999}


<IPython.core.display.Latex object>

### Ejercicio 2
A partir del código proporcionado crea el equivalente a una puerta AND de tal forma que Q0 = Q1 AND Q2.

Estado esperado: $\sqrt(0.25)|000\rangle + \sqrt(0.25)|010\rangle + \sqrt(0.25)|100\rangle + \sqrt(0.25)|111\rangle$

In [9]:
state = Statevector.from_int(0, 2**3)
qc = QuantumCircuit(3)
qc.h(1)
qc.h(2)
#========================

# Aplicar la puerta Toffoli (ccx) con Q1 y Q2 como controles y Q0 como objetivo
qc.ccx(1, 2, 0)

#========================
state = state.evolve(qc)
print(qc)
print(state.probabilities_dict())
state.draw('latex')

          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     ├───┤  │  
q_2: ┤ H ├──■──
     └───┘     
{'000': 0.2499999999999999, '010': 0.2499999999999999, '100': 0.2499999999999999, '111': 0.2499999999999999}


<IPython.core.display.Latex object>

### Ejercicio 3
A partir del código proporcionado crea el equivalente a una puerta OR de tal forma que Q0 = Q1 OR Q2.

Estado esperado: $\sqrt(0.25)|000\rangle + \sqrt(0.25)|011\rangle + \sqrt(0.25)|101\rangle + \sqrt(0.25)|111\rangle$

In [10]:
state = Statevector.from_int(0, 2**3)
qc = QuantumCircuit(3)
qc.h(1)
qc.h(2)
#========================

# Aplicar la puerta X a Q1 y Q2 para invertir sus estados
qc.x(1)
qc.x(2)

# Aplicar la puerta Toffoli (ccx) con Q1 y Q2 como controles y Q0 como objetivo
qc.ccx(1, 2, 0)

# Aplicar la puerta X a Q1 y Q2 nuevamente para revertir la inversión inicial
qc.x(1)
qc.x(2)

#========================
state = state.evolve(qc)
print(qc)
print(state.probabilities_dict())
state.draw('latex')

               ┌───┐     
q_0: ──────────┤ X ├─────
     ┌───┐┌───┐└─┬─┘┌───┐
q_1: ┤ H ├┤ X ├──■──┤ X ├
     ├───┤├───┤  │  ├───┤
q_2: ┤ H ├┤ X ├──■──┤ X ├
     └───┘└───┘     └───┘
{'001': 0.2499999999999999, '010': 0.2499999999999999, '100': 0.2499999999999999, '110': 0.2499999999999999}


<IPython.core.display.Latex object>

### Ejercicio 4
A partir del estado proporcionado $Ψ = |00\rangle + |11\rangle$ utiliza las puertas necesarias para obtener el estado $|00\rangle$ con probabilidad 100%.

In [11]:
state = Statevector([1/math.sqrt(2)+0.j,0.+0.j, 0.+0.j,1/math.sqrt(2)+0.j])
qc = QuantumCircuit(2)
#========================

# Aplicar la puerta CNOT con el primer qubit como control y el segundo como objetivo
qc.cx(0, 1)

# Aplicar la puerta Hadamard al primer qubit
qc.h(0)

#========================
state = state.evolve(qc)
print(qc)
print( {key: f'{value:.2f}' for key, value in state.probabilities_dict().items() if value > 0.01}) # Ajustar resultado debido a imprecisiones con decimales
state.draw('latex')

          ┌───┐
q_0: ──■──┤ H ├
     ┌─┴─┐└───┘
q_1: ┤ X ├─────
     └───┘     
{'00': '1.00'}


<IPython.core.display.Latex object>

### Ejercicio 5
El qubit 0 tiene 20% de probabilidad de $|0\rangle$ y 80% de $|1\rangle$ y el qubit 1 tiene 100% de probabilidad de $|0\rangle$

Intercambialos para obtener $\sqrt(0.2)|00\rangle + \sqrt(0.8)|01\rangle$



In [ ]:
# Aplicar las amplitudes dadas a los estados |00> y |10>
state = Statevector([math.sqrt(0.2), 0, math.sqrt(0.8), 0])
qc = QuantumCircuit(2)
#========================



#========================
state = state.evolve(qc)
print(qc)
print( {key: f'{value:.2f}' for key, value in state.probabilities_dict().items() if value > 0.01}) # Ajustar resultado debido a imprecisiones con decimales
state.draw('latex')

     
q_0: 
     
q_1: 
     
{'00': '0.20', '10': '0.80'}


<IPython.core.display.Latex object>

### Ejercicio 6
A partir del estado proporcionado $Ψ = \sqrt(0.2)|00\rangle + \sqrt(0.4)|10\rangle + \sqrt(0.4)|11\rangle$ utiliza las puertas necesarias para obtener el estado $\sqrt(0.2)|00\rangle + \sqrt(0.8)|11\rangle$

In [12]:
state = Statevector([math.sqrt(0.2), 0, math.sqrt(0.4), math.sqrt(0.4)])
qc = QuantumCircuit(2)
#========================

# No hay que hacer nada no?

#========================
state = state.evolve(qc)
print(qc)
print( {key: f'{value:.2f}' for key, value in state.probabilities_dict().items() if value > 0.01}) # Ajustar resultado debido a imprecisiones con decimales
state.draw('latex')

     
q_0: 
     
q_1: 
     
{'00': '0.20', '10': '0.40', '11': '0.40'}


<IPython.core.display.Latex object>

### Ejercicio 7
A partir del estado proporcionado $|10⟩$ utiliza las puertas necesarias para obtener el estado $|11\rangle$ sin modificar las puertas Hadamard controladas al inicio y al final del circuito.

In [13]:
state = Statevector([0,1,0,0])
qc = QuantumCircuit(2)
qc.ch(0,1)
#========================

qc.x(1)


#========================
qc.ch(1,0)
state = state.evolve(qc)
print(qc)
print( {key: f'{value:.2f}' for key, value in state.probabilities_dict().items() if value > 0.01}) # Ajustar resultado debido a imprecisiones con decimales
state.draw('latex')

               ┌───┐
q_0: ──■───────┤ H ├
     ┌─┴─┐┌───┐└─┬─┘
q_1: ┤ H ├┤ X ├──■──
     └───┘└───┘     
{'01': '0.50', '10': '0.25', '11': '0.25'}


<IPython.core.display.Latex object>

### Ejercicio 8
Dado el estado $\Psi = \sqrt(0.1)|00\rangle + \sqrt(0.2)|01\rangle + \sqrt(0.3)|10\rangle + \sqrt(0.4)|11\rangle$

Intercambia la probabilidad de obtener $|01\rangle$ y $|10\rangle$

In [14]:
state = Statevector([math.sqrt(0.1),math.sqrt(0.2),math.sqrt(0.3),math.sqrt(0.4)])
qc = QuantumCircuit(2)
#========================

# Aplicar la puerta CNOT para entrelazar los qubits
qc.cx(0, 1)
# Aplicar la puerta CNOT en la dirección opuesta
qc.cx(1, 0)
# Aplicar la puerta CNOT otra vez para deshacer el entrelazamiento inicial
qc.cx(0, 1)

#========================
state = state.evolve(qc)
print(qc)
print( {key: f'{value:.2f}' for key, value in state.probabilities_dict().items() if value > 0.01}) # Ajustar resultado debido a imprecisiones con decimales
state.draw('latex')

          ┌───┐     
q_0: ──■──┤ X ├──■──
     ┌─┴─┐└─┬─┘┌─┴─┐
q_1: ┤ X ├──■──┤ X ├
     └───┘     └───┘
{'00': '0.10', '01': '0.30', '10': '0.20', '11': '0.40'}


<IPython.core.display.Latex object>

## Entrega
Se puede compartir el enlace a la copia personal de cada alumno del cuaderno de Google Colab. Para obtenerlo, se debe pulsar en el botón Compartir (Share), arriba a la derecha, y en el apartado de Obtener enlace, seleccionar la opción de Copiar enlace. Hay que asegurarse de que el enlace está configurado para que cualquiera con el enlace pueda acceder al contenido, y debe compartirse con permisos de Lector.